In [ ]:
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [ ]:
# Ruta a Carpeta de Codigos de Estudiantes y al modelo CodeBERTa
ruta_carpeta = 'path carpeta de ejemplos estudiantes'
ruta_modelo = 'ruta al archivo pytorch_model.bin'

In [ ]:
# Inicialización de DataFrame
data = pd.DataFrame()

In [ ]:
# Cargar el modelo pre-entrenado
model = AutoModelForSequenceClassification.from_pretrained(ruta_modelo)
tokenizer = AutoTokenizer.from_pretrained(ruta_modelo)

In [ ]:
# Recorrer carpeta de codigos

for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.py'):
        # Leer el archivo
        with open(os.path.join(ruta_carpeta, archivo), 'r') as f:
            codigo = f.read()

        # Preprocesamiento de codigo fuente
        for index, row in data.iterrows():
            code = row['codigo']

        # Limpiar código fuente
        codigo = codigo.replace('...', '')  # Eliminar comentarios
        codigo = codigo.replace(' ', ' ')  # Eliminar espacios en blanco

        # Tokenizar código fuente
        tokens = word_tokenize(codigo)

        # Eliminar tokens irrelevantes
        stop_words = set(stopwords.words('spanish'))
        tokens = [token for token in tokens if token not in stop_words]

In [ ]:
# Función de vectorización y almacenamiento
def vectorizar_y_almacenar(tokens_list):
    # Vectorizar el código con CodeBERTa
    representaciones = []

    for tokens in tokens_list:
        inputs = tokenizer(tokens, return_tensors='pt')  # Codificar el código
        with torch.no_grad():
            outputs = model(**inputs)  # Obtener representaciones del modelo
        representaciones.append(outputs.logits.squeeze(0).detach().cpu().numpy())  # Extraer representaciones

    # Almacenar datos en DataFrame
    data_vectorizada = pd.DataFrame(representaciones, columns=model.config.num_labels_to_classify)
    return data_vectorizada

In [ ]:
# Recorrer carpeta de codigos
tokens_list = []

for archivo in os.listdir(ruta_carpeta):
    if archivo.endswith('.py'):
        # Leer el archivo
        with open(os.path.join(ruta_carpeta, archivo), 'r') as f:
            codigo = f.read()

        # Preprocesar el código
        tokens_list.append(tokens)

In [ ]:
# Vectorizar y almacenar
data_vectorizada = vectorizar_y_almacenar(tokens_list)

In [ ]:
# Unir datos con DataFrame principal
data = data.join(data_vectorizada)

In [ ]:
# Visualización de resultados
data.groupby('calificacion')['codigo_preprocesado'].apply(list).head()